In [ ]:
!pip install optuna -q
!gdown 1psrRtIHCtLj19Hfp0gstBoTzSWveClqu

Downloading...
From: https://drive.google.com/uc?id=1psrRtIHCtLj19Hfp0gstBoTzSWveClqu
To: /content/data.csv
100% 4.84M/4.84M [00:00<00:00, 217MB/s]


In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

import optuna

Dungeons & Dragons (D&D, DnD; Подземелья и драконы) — настольная ролевая игра в жанре фэнтези, разработанная Гэри Гайгэксом и Дэйвом Арнесоном.

- https://dnd.wizards.com/
- https://ru.wikipedia.org/wiki/Dungeons_%26_Dragons

# Предсказание расы персонажа по его характеристикам

В данной работе выполняется предсказание расы персонажа Dungeon&Dragons, основываясь на его характеристиках:
- `Strength` - показатель силы, измеряется от 3 до 20
- `Dexterity` - показатель ловкости, измеряется от 3 до 20
- `Constitution` - показатель выносливости, измеряется от 3 до 20
- `Intelligence` - показатель интеллекта, измеряется от 3 до 20
- `Wisdom` - показатель мудрости, измеряется от 3 до 20
- `Charisma` - показатель харизмы, измеряется от 3 до 20
- `Age` - возраст в годах
- `Height` - рост в сантиметрах
- `Weight` - вес в фунтах
- `NightVision` - дальность ночного зрения
- `Size` - описание размера (`small / medium`)


image.png


In [ ]:
df = pd.read_csv('data.csv', index_col='Unnamed: 0')
df

,Strength,Dexterity,Constitution,Intelligence,Wisdom,Charisma,Race,Speed,NightVision,Size,Age,Height,Weight
0,13,9,9,12,12,14,Gnome,25,60,small,261,113,33
1,14,7,16,16,11,16,Gnome,25,60,small,105,104,33
2,9,14,9,14,9,11,Gnome,25,60,small,117,113,37
3,11,17,8,8,12,13,Gnome,25,60,small,61,118,38
4,13,13,13,15,8,14,Gnome,25,60,small,41,115,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,14,16,10,13,14,16,Half-Elf,30,60,medium,81,165,106
89996,13,7,12,14,4,17,Half-Elf,30,60,medium,84,169,110
89997,13,8,13,15,12,16,Half-Elf,30,60,medium,65,175,119
89998,16,14,15,14,17,17,Half-Elf,30,60,medium,83,170,127


Изменим target (Race) на числовые значения и отбросим столбец Size, так как по сути информацию о размере мы получаем из Height и Weight.

In [ ]:
races = {race: i for i, race in enumerate(df['Race'].unique())}
races

{'Gnome': 0,
 'Dwarf': 1,
 'Elf': 2,
 'Halfling': 3,
 'Human': 4,
 'Tiefling': 5,
 'Dragonborn': 6,
 'Half-Orc': 7,
 'Half-Elf': 8}

In [ ]:
df['Race'] = df['Race'].apply(lambda x: races[x])

In [ ]:
X, y = df.drop(['Race', 'Size'], axis=1), df['Race']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=y_train, name='Train'))
fig.add_trace(go.Histogram(x=y_test, name='Test'))
fig.update_layout(
    title_text='Number of objects in the partition',
    xaxis_title_text='Race',
    yaxis_title_text='Count',)
fig.show()

Будем использовать Случайный лес, так как в данных нет какой-либо линейной зависимости и целью является многоклассовая классификация

## RandomForest

### Подбор гиперпараметров

Найдем наилучшие гиперпараметры с помощью `optuna`

In [ ]:
def find_params(X, y, n=10):
  def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
    n_est = trial.suggest_int('n_est', 10, 500)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    clf = RandomForestClassifier(
        n_estimators=n_est,
        criterion=criterion,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features)
    clf.fit(X_train, y_train)
    return clf.score(X_test, y_test)
  study = optuna.create_study(direction='maximize')
  study.optimize(objective, n_trials=n, show_progress_bar=True)
  return study

In [ ]:
find_params(df.drop(['Race', 'Size'], axis=1), df['Race'], 50)

[I 2023-06-09 15:46:05,876] A new study created in memory with name: no-name-a5c4ba36-fb62-480c-9aa5-f9e12dc76729


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-06-09 15:46:39,079] Trial 0 finished with value: 0.9667111111111111 and parameters: {'n_est': 456, 'criterion': 'gini', 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 0 with value: 0.9667111111111111.
[I 2023-06-09 15:47:41,549] Trial 1 finished with value: 0.9668444444444444 and parameters: {'n_est': 462, 'criterion': 'gini', 'min_samples_leaf': 7, 'max_features': None}. Best is trial 1 with value: 0.9668444444444444.
[I 2023-06-09 15:48:22,893] Trial 2 finished with value: 0.9672 and parameters: {'n_est': 289, 'criterion': 'gini', 'min_samples_leaf': 3, 'max_features': None}. Best is trial 2 with value: 0.9672.
[I 2023-06-09 15:49:22,417] Trial 3 finished with value: 0.9676 and parameters: {'n_est': 435, 'criterion': 'log_loss', 'min_samples_leaf': 4, 'max_features': None}. Best is trial 3 with value: 0.9676.
[I 2023-06-09 15:49:54,950] Trial 4 finished with value: 0.9678222222222223 and parameters: {'n_est': 475, 'criterion': 'entropy', 'min_samples_leaf': 10

### Оценка классификации

Теперь создадим с полученными значениями классификатор и рассмотрим закономерности в результатах

In [ ]:
X, y = df.drop(['Race', 'Size'], axis=1), df['Race']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

clf = RandomForestClassifier(
      n_estimators=400,
      criterion='entropy',
      min_samples_leaf=3,
      max_features='log2')

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
races_reverse = {races[race]:race for race in races}

y_pred_names = [races_reverse[race] for race in y_pred]
y_test_names = [races_reverse[race] for race in y_test]

print(classification_report(y_pred_names, y_test_names))

              precision    recall  f1-score   support

  Dragonborn       1.00      1.00      1.00      2496
       Dwarf       1.00      1.00      1.00      2500
         Elf       0.97      0.91      0.94      2654
       Gnome       1.00      1.00      1.00      2500
    Half-Elf       0.83      0.88      0.85      2357
    Half-Orc       0.92      0.92      0.92      2489
    Halfling       1.00      1.00      1.00      2500
       Human       1.00      1.00      1.00      2504
    Tiefling       1.00      1.00      1.00      2500

    accuracy                           0.97     22500
   macro avg       0.97      0.97      0.97     22500
weighted avg       0.97      0.97      0.97     22500



In [ ]:
mistakes = {race: [] for race in races}

for i, real in enumerate(y_test_names):
  mistakes[real].append(y_pred_names[i])

fig = make_subplots(rows=3, cols=3)

for i, race in enumerate(races):
  fig.append_trace(go.Histogram(x=mistakes[race], name=race), i // 3 + 1, i % 3 + 1)

fig.update_layout(
    title_text='Classification results',
    )

fig.show()

In [ ]:
feature_names = X.columns
importances = clf.feature_importances_

fig = px.bar(x=feature_names, y=importances, title='Feature importances')
fig.show()

### Эксперимент

Как видно, способности (Сила, Ловкость и т.д.) не сильно влияют на результат. Для эксперимента, попробуем отбросить всё, кроме них.

In [ ]:
find_params(df.drop(['Race', 'Size', 'Speed', 'NightVision', 'Age', 'Height', 'Weight'], axis=1), df['Race'])

[I 2023-06-09 16:11:48,494] A new study created in memory with name: no-name-cc547c21-f2f3-48c3-b016-6c478cde3154


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-06-09 16:12:05,584] Trial 0 finished with value: 0.21208888888888888 and parameters: {'n_est': 177, 'criterion': 'log_loss', 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.21208888888888888.
[I 2023-06-09 16:13:06,270] Trial 1 finished with value: 0.21351111111111112 and parameters: {'n_est': 328, 'criterion': 'entropy', 'min_samples_leaf': 6, 'max_features': None}. Best is trial 1 with value: 0.21351111111111112.
[I 2023-06-09 16:13:09,283] Trial 2 finished with value: 0.17942222222222223 and parameters: {'n_est': 20, 'criterion': 'entropy', 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.21351111111111112.
[I 2023-06-09 16:13:28,064] Trial 3 finished with value: 0.2248888888888889 and parameters: {'n_est': 234, 'criterion': 'log_loss', 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 3 with value: 0.2248888888888889.
[I 2023-06-09 16:14:15,452] Trial 4 finished with value: 0.1834222222222222 and parame

In [ ]:
X, y = df.drop(['Race', 'Size', 'Speed', 'NightVision', 'Age', 'Height', 'Weight'], axis=1), df['Race']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

clf = RandomForestClassifier(
      n_estimators=200,
      criterion='log_loss',
      min_samples_leaf=10,
      max_features='sqrt')

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
y_pred_names = [races_reverse[race] for race in y_pred]
y_test_names = [races_reverse[race] for race in y_test]

print(classification_report(y_pred_names, y_test_names))

              precision    recall  f1-score   support

  Dragonborn       0.24      0.23      0.24      2649
       Dwarf       0.37      0.26      0.30      3637
         Elf       0.19      0.20      0.20      2343
       Gnome       0.34      0.25      0.28      3410
    Half-Elf       0.24      0.25      0.25      2417
    Half-Orc       0.22      0.22      0.22      2540
    Halfling       0.21      0.20      0.20      2536
       Human       0.04      0.12      0.05       754
    Tiefling       0.19      0.21      0.20      2214

    accuracy                           0.23     22500
   macro avg       0.23      0.22      0.22     22500
weighted avg       0.25      0.23      0.24     22500



In [ ]:
mistakes = {race: [] for race in races}

for i, real in enumerate(y_test_names):
  mistakes[real].append(y_pred_names[i])

fig = make_subplots(rows=3, cols=3)

for i, race in enumerate(races):
  fig.append_trace(go.Histogram(x=mistakes[race], name=race), i // 3 + 1, i % 3 + 1)

fig.update_layout(
    title_text='Classification results',
    )

fig.show()

Как и предполагалось в первой части, данные признаки не очень информативны для классификации. Однако на графиках выше видно, что некоторые расы все-таки получается хоть сколько-то часто угадывать. Сложнее всего распознать человека, что вполне обусловлено правилами Dungeon&Dragons, где "человек" может быть развит в чём угодно.

### Эксперимент 2

Если же эти признаки не так информативны, может, есть смысл их вообще не использовать?

In [ ]:
find_params(df.drop(['Race', 'Size', 'Strength', 'Dexterity', 'Constitution', 'Intelligence', 'Wisdom', 'Charisma'], axis=1), df['Race'])

[I 2023-06-09 16:18:53,427] A new study created in memory with name: no-name-998eed0d-9baa-451c-bd07-62c5b1b4456a


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-06-09 16:19:00,156] Trial 0 finished with value: 0.9584 and parameters: {'n_est': 123, 'criterion': 'gini', 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.9584.
[I 2023-06-09 16:19:04,066] Trial 1 finished with value: 0.9554222222222222 and parameters: {'n_est': 85, 'criterion': 'log_loss', 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 0 with value: 0.9584.
[I 2023-06-09 16:19:30,516] Trial 2 finished with value: 0.9564 and parameters: {'n_est': 374, 'criterion': 'entropy', 'min_samples_leaf': 5, 'max_features': None}. Best is trial 0 with value: 0.9584.
[I 2023-06-09 16:19:44,856] Trial 3 finished with value: 0.9567555555555556 and parameters: {'n_est': 182, 'criterion': 'log_loss', 'min_samples_leaf': 3, 'max_features': None}. Best is trial 0 with value: 0.9584.
[I 2023-06-09 16:19:50,584] Trial 4 finished with value: 0.9561777777777778 and parameters: {'n_est': 100, 'criterion': 'log_loss', 'min_samples_leaf': 2, 'max_features':

In [ ]:
X, y = df.drop(['Race', 'Size', 'Strength', 'Dexterity', 'Constitution', 'Intelligence', 'Wisdom', 'Charisma'], axis=1), df['Race']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

clf = RandomForestClassifier(
      n_estimators=150,
      criterion='log_loss',
      max_features='log2',
      min_samples_leaf=8)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
y_pred_names = [races_reverse[race] for race in y_pred]
y_test_names = [races_reverse[race] for race in y_test]

print(classification_report(y_pred_names, y_test_names))

              precision    recall  f1-score   support

  Dragonborn       1.00      1.00      1.00      2499
       Dwarf       1.00      1.00      1.00      2500
         Elf       0.95      0.88      0.91      2710
       Gnome       1.00      1.00      1.00      2500
    Half-Elf       0.79      0.83      0.81      2357
    Half-Orc       0.89      0.91      0.90      2433
    Halfling       1.00      1.00      1.00      2500
       Human       1.00      1.00      1.00      2501
    Tiefling       1.00      1.00      1.00      2500

    accuracy                           0.96     22500
   macro avg       0.96      0.96      0.96     22500
weighted avg       0.96      0.96      0.96     22500



Таким образом, несмотря на слабую значимость, способности все-таки помогают определять расу персонажа лучше.